In [1]:
import torch
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import dataset

from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer



In [3]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


device = try_gpu(0)

# Preprocessing Text

In [3]:
import regex as re


#Data path to ne_dedup.txt
datapath = 'F:/NLP-project-junk/NLP/Nepali-Language-Processing/data/ne_dedup.txt'

with open(datapath, 'r', encoding='utf-8') as f:
    text = f.read()
    
    
    #Add space to the right places
    text = re.sub(r'\s*[\u0964]\s*', r'\u0020\u0964\u0020', text)
    text = re.sub(r'\s*[\u003f]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*[\u002c]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*\n\s*','\n', text)
    #text = re.sub(r'\s*[\u0966-\u0976]+\s*','\u0020[\u0966-\u0976]\u0020', text)
    #text = re.sub(r'\s+?\s+', r'\u0020?\u0020', text)
    text = re.sub(r'[^\u0900-\u097F,?\s+]','', text)
    
    

In [4]:
c = re.sub(r'[^\u0900-\u097F,?]','','?नमस्तेnishant,')
#c = re.sub(r'\s*[\u002c]\s*','\u0020\u002c\u0020','?नमस्तेnishant,')
c


'?नमस्ते,'

In [5]:
hex(ord('०'))

'0x966'

In [6]:
len(text.split('\n'))

341961

In [9]:
train_iter_first = text.split('\n')[:30000]
train_iter_second = text.split('\n')[:35000]
test_iter = text.split('\n')[35000:40000]

In [6]:
train_iter_first = text.split('\n')[:300000]
# train_iter_second = text.split('\n')[:35000]
test_iter = text.split('\n')[300000:]

# Run Here

In [8]:
from tokenizers import ByteLevelBPETokenizer

In [9]:
help(ByteLevelBPETokenizer)

Help on class ByteLevelBPETokenizer in module tokenizers.implementations.byte_level_bpe:

class ByteLevelBPETokenizer(tokenizers.implementations.base_tokenizer.BaseTokenizer)
 |  ByteLevelBPETokenizer(vocab: Union[str, Dict[str, int], NoneType] = None, merges: Union[str, Dict[Tuple[int, int], Tuple[int, int]], NoneType] = None, add_prefix_space: bool = False, lowercase: bool = False, dropout: Optional[float] = None, unicode_normalizer: Optional[str] = None, continuing_subword_prefix: Optional[str] = None, end_of_word_suffix: Optional[str] = None, trim_offsets: bool = False)
 |  
 |  ByteLevelBPETokenizer
 |  
 |  Represents a Byte-level BPE as introduced by OpenAI with their GPT-2 model
 |  
 |  Method resolution order:
 |      ByteLevelBPETokenizer
 |      tokenizers.implementations.base_tokenizer.BaseTokenizer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, vocab: Union[str, Dict[str, int], NoneType] = None, merges: Union[str, Dict[Tuple[int, int], Tupl

In [10]:
tokenizer = ByteLevelBPETokenizer()

In [12]:
help(tokenizer.train_from_iterator)

Help on method train_from_iterator in module tokenizers.implementations.byte_level_bpe:

train_from_iterator(iterator: Union[Iterator[str], Iterator[Iterator[str]]], vocab_size: int = 30000, min_frequency: int = 2, show_progress: bool = True, special_tokens: List[Union[str, tokenizers.AddedToken]] = [], length: Optional[int] = None) method of tokenizers.implementations.byte_level_bpe.ByteLevelBPETokenizer instance
    Train the model using the given iterator



In [142]:
# tokenizer.train_from_iterator(train_iter_first,vocab_size = 30000,min_frequency = 2,special_tokens = []) #,'।'

In [209]:
# tokenizer.train(files='F:/NLP-project-junk/NLP/Nepali-Language-Processing/data/ne_dedup.txt', vocab_size=30000, min_frequency=2)

In [11]:
import pickle

In [12]:
#

In [13]:
# with open('tokenizer.pickle','wb') as f:
#     pickle.dump(tokenizer,f)

In [12]:
with open('tokenizer.pickle','rb') as f:
    t = pickle.load(f)

In [ ]:
#

In [143]:
# with open('tokenizer_full2.pickle','wb') as f:
#     pickle.dump(tokenizer,f)

In [14]:
with open('tokenizer_full2.pickle','rb') as f:
    t = pickle.load(f)

In [15]:
# tokenizer = t

In [253]:
# tokenizer.save("vocab.json")

In [146]:
# tokenizer.save("vocab_full2.json")

In [16]:
vocab = tokenizer.get_vocab()
len(vocab)

30000

In [17]:
encoded = t.encode("विकास")
print(encoded.tokens)
decoded = t.decode(encoded.ids)
print(decoded)

['à¤µ', 'à¤¿', 'à¤ķ', 'à¤¾', 'à¤¸']
विकास


In [14]:
tokenizer.decode(tokenizer.encode('उनको').ids)

'उनको'

In [257]:
print(len(train_iter_first),len(train_iter_second),len(test_iter))

30000 35000 5000


In [3]:
# import pickle


# tokenizer = get_tokenizer(None)

# # vocab = build_vocab_from_iterator(
# #     map(tokenizer, train_iter_first), specials=['<unk>'])
# # vocab.set_default_index(vocab['<unk>'])


# # Save for first time
# # with open('transformer_vocab.pickle','wb') as f:
# #     pickle.dump(vocab,f)

# with open('transformer_vocab.pickle','rb') as f:
#     vocab = pickle.load(f)

In [16]:
tokenizer = t

In [20]:
l = tokenizer.encode('महानायक राजेश हमाल अहिले चलचित्र क्षेत्रमा पातलिए ।').ids
l

[1021,
 259,
 262,
 259,
 336,
 285,
 259,
 293,
 265,
 309,
 1647,
 259,
 267,
 468,
 264,
 267,
 265,
 721,
 264,
 269,
 260,
 261,
 282,
 260,
 299,
 265,
 269,
 260,
 328,
 259,
 276,
 259,
 524,
 264,
 290,
 281]

In [26]:
tokenizer.decode(l)
# tokenizer.encode('।').ids

'महानायकराजेशहमालअहिलेचलचित्रक्षेत्रमापातलिए।'

In [152]:
tokenizer.decode([258,98])

' थ'

In [18]:
def split_list(l):
    splitted_list = []
    z = 0
    for i,idx in enumerate(l):
        if idx == 220:
            splitted_list.append(l[z:i])
            z = i+1
    if z <= len(l)-1:
        splitted_list.append(l[z:])
    return splitted_list

In [21]:
k = split_list(l)
k

[[1021,
  259,
  262,
  259,
  336,
  285,
  259,
  293,
  265,
  309,
  1647,
  259,
  267,
  468,
  264,
  267,
  265,
  721,
  264,
  269,
  260,
  261,
  282,
  260,
  299,
  265,
  269,
  260,
  328,
  259,
  276,
  259,
  524,
  264,
  290,
  281]]

In [22]:
def splits_to_token(splited_list):
    strings = [tokenizer.decode(l) for l in splited_list]
    
    return strings
    

In [159]:
splits_to_token(k)

['']

In [21]:
len('महानायक राजेश हमाल अहिले चलचित्र क्षेत्रमा पातलिए ।')

51

In [241]:
tokenizer.encode(' ').ids

[0]

In [242]:
'aa '

'aa '

In [23]:
tokenizer.decode([978,
 261,
 264,
 261,
 263])

'डररिरक'

In [24]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(tokenizer.encode(item).ids, dtype=torch.long)
            for item in raw_text_iter]
#     print("--aaaaaaaaaa",data)
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [29]:
#device = torch.device('cpu')
train_data = data_process(train_iter_second[:10])

In [288]:
tokenizer.encode(' ').ids

[0]

In [164]:
#Train and Test Split
# train_data = data_process(train_iter_second)
train_data = data_process(train_iter_first)
test_data = data_process(test_iter)

In [165]:
len(train_data)

273494001

In [38]:
train_data[:400]

tensor([  534,   261,   280,   265,   299,   260,   274,     0,   947,   260,
          268,   265,   264,   260,   264,   267,     0,   270,   266,   274,
          261,   262,   267,     0,   771,     0,   360,   265,  4743,   260,
          288,     0,   279,   260,   262,   265,   270,     0,  1700,  3157,
            0,   258,    98,     0,  1395,     0,   264,   267,     0,   274,
          312,   310,   267,   302,   265,   270,     0,   262,     0,  3238,
            0,   258,    98,     0,  1307,     0,   264,   267,     0,   279,
          261,   395,   261,   270,   260,   281,   265,   270,     0,   263,
          273,   270,   265,     0,    31,     0,   264,   260,   262,   261,
          337,   261,   329,     0,  3956,   260,     0,  1299,   266,   288,
          436,   260,   262,   261,   307,   265,   264,     0,   434,   261,
          300,     0, 13276,     0,   264,   598,     0,   970,   265,   355,
          261,   271,     0,   821,   260,   262,   261,   973, 

In [39]:
print(len(vocab))

30000


In [166]:

# print(len(train_iter_second),len(test_iter))
torch.cuda.empty_cache() 

torch.cuda.memory_allocated() 

288843776

In [167]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
#     data = data.view(bsz, seq_len).t().contiguous()
    data = data.view(bsz,seq_len).t()
#     return data.to(device)
    return data

In [168]:
# batch_size = 35
# eval_batch_size = 10

# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# test_data = batchify(test_data, eval_batch_size)

# Working with a dummy Sample

In [66]:
#Sample Data


text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [67]:
sample_data

tensor([3520,  264,  265,  260,  262,  264,  265,    0,  263,  264,  262,  261,
        2132,  267,    0,  265,  260,  868,  266,    0,   12,    0,  281,  260,
         318,  264,  295,  261,  271,    0,  299,  264,  321,  260,  504,  266,
           0,   12,    0,  274,  310,  271,  272,  265,  261,  270,    0,  262,
         260,  295,  261,  271,    0, 2303,  266,  262,  264,  265,  273,    0,
         359,  272,  280,  261,  262,  273,    0,  279,  260,  263,  273,    0,
         263,  264,  262,  261,  269,  260,  270,  260,  280,  261,  281,  260,
         262,  260,    0,  274,  310,  271,  272,  265,  261,  270,    0,  262,
         260,  295,  261,  271])

In [68]:
sample_data = batchify(sample_data, 2)
print("Given word:", text[0])
sample_data

Given word: आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य


tensor([[3520,  261],
        [ 264,  271],
        [ 265,    0],
        [ 260, 2303],
        [ 262,  266],
        [ 264,  262],
        [ 265,  264],
        [   0,  265],
        [ 263,  273],
        [ 264,    0],
        [ 262,  359],
        [ 261,  272],
        [2132,  280],
        [ 267,  261],
        [   0,  262],
        [ 265,  273],
        [ 260,    0],
        [ 868,  279],
        [ 266,  260],
        [   0,  263],
        [  12,  273],
        [   0,    0],
        [ 281,  263],
        [ 260,  264],
        [ 318,  262],
        [ 264,  261],
        [ 295,  269],
        [ 261,  260],
        [ 271,  270],
        [   0,  260],
        [ 299,  280],
        [ 264,  261],
        [ 321,  281],
        [ 260,  260],
        [ 504,  262],
        [ 266,  260],
        [   0,    0],
        [  12,  274],
        [   0,  310],
        [ 274,  271],
        [ 310,  272],
        [ 271,  265],
        [ 272,  261],
        [ 265,  270],
        [ 261,    0],
        [ 

In [111]:
j = get_batch(sample_data,1)
j

(tensor([[ 264,  271],
         [ 265,    0],
         [ 260, 2303],
         [ 262,  266],
         [ 264,  262],
         [ 265,  264],
         [   0,  265],
         [ 263,  273],
         [ 264,    0],
         [ 262,  359],
         [ 261,  272],
         [2132,  280],
         [ 267,  261],
         [   0,  262],
         [ 265,  273],
         [ 260,    0],
         [ 868,  279],
         [ 266,  260],
         [   0,  263],
         [  12,  273],
         [   0,    0],
         [ 281,  263],
         [ 260,  264],
         [ 318,  262],
         [ 264,  261],
         [ 295,  269],
         [ 261,  260],
         [ 271,  270],
         [   0,  260],
         [ 299,  280],
         [ 264,  261],
         [ 321,  281],
         [ 260,  260],
         [ 504,  262],
         [ 266,  260],
         [   0,    0],
         [  12,  274],
         [   0,  310],
         [ 274,  271],
         [ 310,  272],
         [ 271,  265],
         [ 272,  261],
         [ 265,  270],
         [ 

In [113]:
j[0].shape

torch.Size([48, 2])

In [114]:
j[1].shape

torch.Size([96])

In [31]:
train_data.shape

torch.Size([39879286])

In [80]:
train_data.to('cpu').shape
test_data.to('cpu').shape


torch.Size([51254056])

In [81]:
train_data.device

device(type='cpu')

device(type='cpu')

In [169]:
bptt = 16
train_data_batched = batchify(train_data, bptt)  # shape [seq_len, batch_size]
test_data_batched = batchify(test_data, bptt)

In [162]:
torch.cuda.empty_cache()

In [163]:
train_data_batched.shape

torch.Size([21387439, 16])

In [92]:
j[1].shape

torch.Size([4096])

In [31]:

seq_length = 128
import math
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(seq_length, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

In [48]:
j = get_batch(train_data_batched,2)

In [51]:
j[0].shape

torch.Size([128, 32])

# Model Definition

In [33]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)





class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(
            d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [172]:
# torch.cuda.memory_stats()


In [34]:
torch.cuda.memory_allocated()

0

In [34]:
#device = torch.device('cpu')

In [174]:
seq_length = 128

# Hyper-Parameter Tuning

In [35]:
import math

In [36]:
ntokens = len(vocab)  # size of vocabulary
emsize = 300  # embedding dimension
d_hid = 800  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # number of heads in nn.MultiheadAttention
dropout = 0.05  # dropout probability
model = TransformerModel(ntokens, emsize,nhead, d_hid,
                         nlayers, dropout).to(device)

In [177]:
criterion = nn.CrossEntropyLoss()
lr = 1  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [53]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [52]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 800
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(seq_length).to(device)

    num_batches = len(train_data_batched) // seq_length
    for batch, i in enumerate(range(0, train_data_batched.size(0) - 1, seq_length)):
        data, targets = get_batch(train_data_batched, i)
        batch_size = data.size(0)
        if batch_size != seq_length:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data.to(device), src_mask)
#         print(output.shape,targets.shape,data.shape)
        #print(type(output)
        loss = criterion(output.view(-1, ntokens), targets.to(device))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()
        if batch % 4000 == 0 and batch>0:
            print("Checkpointing...")
            torch.save(model.state_dict(),'checkpoint2.pt')
            

In [70]:
softmax = nn.Softmax(dim=2)
#softmax = nn.LogSoftmax(dim=2)

In [50]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(seq_length).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, seq_length):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != seq_length:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data.to(device), src_mask)
            output_softmax = softmax(output)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            # print(output)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets.to(device)).item()

    return total_loss / (len(eval_data) - 1)


In [182]:
#            print('data')
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in data.t()]))
#             print(indices)
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in indices]))
#             print(len(targets))
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in target_indices]))

In [183]:
        # target_indices = targets.t()
        # print(indices.shape)
        # print(indices)
        # temp_gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0][i]
        # gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0]
        # print(temp_text)
        # print([[vocab.lookup_tokens(list(index))
        #        for index in indices][0][i]])
#         temp_text = [[vocab.lookup_tokens(list(index))]
#                       for index in indices[0][i]]
        # print(temp_text)
#         temp_text = [' '.join(temp_text)]
        # print(temp_text)
        # gen_data = vocab.lookup_tokens((list(gen_data))) + list(temp_gen_data)
        # gen_data = torch.tensor(
        #    (list(gen_data[0])+list(indices[0][i]))).unsqueeze(0)
        # gen_data = torch.concat([gen_data, torch.tensor(indices[0][i])], dim=1)
        #gen_data = ' '.join(gen_data)
        #return temp_txt
#         gen_data = data_process(temp_text)
#         gen_data = batchify(gen_data, 1)
        #return gen_data

In [184]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Training Data

In [188]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
import time
import copy
best_val_loss = float('inf')
epochs = 1
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    eval_loss = evaluate(model, test_data_batched)
    eval_ppl = math.exp(eval_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
         f'valid loss {eval_loss:5.2f} | valid ppl {eval_ppl:8.2f}')
    print('-' * 89)
    #best_model = copy.deepcopy(model)
    if eval_loss < best_val_loss:
       best_val_loss = eval_loss
       best_model = copy.deepcopy(model)

#     scheduler.step()
#save model 
#torch.save(best_model.state_dict(),'best_model_3bigx3.pt')
torch.save(best_model.state_dict(),'best_model_3bigx3_corrected_full2.pt')
torch.save(model.state_dict(),'training_model_3bigx3_corrected_full2.pt')


| epoch   1 |   800/133541 batches | lr 1.00 | ms/batch 101.51 | loss  4.04 | ppl    56.97
| epoch   1 |  1600/133541 batches | lr 1.00 | ms/batch 100.86 | loss  3.26 | ppl    26.00
| epoch   1 |  2400/133541 batches | lr 1.00 | ms/batch 100.92 | loss  2.84 | ppl    17.04
| epoch   1 |  3200/133541 batches | lr 1.00 | ms/batch 102.69 | loss  2.64 | ppl    14.05
| epoch   1 |  4000/133541 batches | lr 1.00 | ms/batch 103.22 | loss  2.53 | ppl    12.55
Checkpointing...
| epoch   1 |  4800/133541 batches | lr 1.00 | ms/batch 106.35 | loss  2.43 | ppl    11.41
| epoch   1 |  5600/133541 batches | lr 1.00 | ms/batch 103.76 | loss  2.35 | ppl    10.53
| epoch   1 |  6400/133541 batches | lr 1.00 | ms/batch 104.95 | loss  2.33 | ppl    10.30
| epoch   1 |  7200/133541 batches | lr 1.00 | ms/batch 105.53 | loss  2.30 | ppl     9.98
| epoch   1 |  8000/133541 batches | lr 1.00 | ms/batch 105.70 | loss  2.26 | ppl     9.60
Checkpointing...
| epoch   1 |  8800/133541 batches | lr 1.00 | ms/batch 

| epoch   1 | 70400/133541 batches | lr 1.00 | ms/batch 104.31 | loss  1.87 | ppl     6.50
| epoch   1 | 71200/133541 batches | lr 1.00 | ms/batch 103.87 | loss  1.89 | ppl     6.60
| epoch   1 | 72000/133541 batches | lr 1.00 | ms/batch 103.97 | loss  1.87 | ppl     6.51
Checkpointing...
| epoch   1 | 72800/133541 batches | lr 1.00 | ms/batch 106.63 | loss  1.88 | ppl     6.58
| epoch   1 | 73600/133541 batches | lr 1.00 | ms/batch 103.90 | loss  1.88 | ppl     6.53
| epoch   1 | 74400/133541 batches | lr 1.00 | ms/batch 104.29 | loss  1.87 | ppl     6.47
| epoch   1 | 75200/133541 batches | lr 1.00 | ms/batch 103.79 | loss  1.86 | ppl     6.45
| epoch   1 | 76000/133541 batches | lr 1.00 | ms/batch 103.97 | loss  1.87 | ppl     6.51
Checkpointing...
| epoch   1 | 76800/133541 batches | lr 1.00 | ms/batch 107.48 | loss  1.89 | ppl     6.62
| epoch   1 | 77600/133541 batches | lr 1.00 | ms/batch 104.65 | loss  1.87 | ppl     6.48
| epoch   1 | 78400/133541 batches | lr 1.00 | ms/batch 

KeyboardInterrupt: 

In [181]:
torch.save(best_model.state_dict(),'best_model_3bigx3_corrected.pt')
torch.save(model.state_dict(),'training_model_3bigx3_corrected.pt')

In [190]:
# torch.save(best_model.state_dict(),'best_model_3bigx3_corrected_full2.pt')
torch.save(model.state_dict(),'training_model_3bigx3_corrected_full2.pt')

In [131]:
lnsoftmax = nn.LogSoftmax(dim=2)

In [132]:
model.load_state_dict(torch.load('best_model_3bigx3_corrected.pt'))

<All keys matched successfully>

In [57]:

def id_to_token(x):
    token_list = [tokenizer.id_to_token(id) for id in x]
    
    return token_list

In [183]:
id_to_token([10,15])

['*', '/']

# Data Generation

In [46]:
import time
def generator(model: nn.Module, gen_data: Tensor, no_words = 10):
    model.eval()
#     temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        print('i:', i)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        else:
            src_mask_ = src_mask[:,:]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        print(indices.shape)
        #print(indices[0],indices[1])
        #for j in range(batch_size):
        print('next word: ', [id_to_token(list(index))
                                  for index in indices][0][-1])
        print(i,"Gen_data: ",gen_data,"Pred_data: ",indices)
#         pred_text.append([tokenizer.decode(list(index))
#                                   for index in indices][0][-1])
        pred_text.append(indices[0][-1])
        if(batch_size < 128):
            gen_data = torch.cat((gen_data[:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
            
    return pred_text



In [192]:
st = ['म भारत भ्रमण गर्न']
st_i = data_process(st)

st_i = st_i.unsqueeze(1).to(device)

In [61]:
st_i.shape

torch.Size([13, 1])

In [62]:
tokenizer.encode(' ').ids

[0]

In [63]:
st_i

tensor([[269],
        [  0],
        [321],
        [260],
        [347],
        [  0],
        [321],
        [261],
        [693],
        [  0],
        [380],
        [261],
        [263]], device='cuda:0')

In [64]:
lnsoftmax = nn.LogSoftmax(dim=2)

In [38]:



bptt = 35
def probability(model: nn.Module, sent: Tensor):
    model.eval()
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    
    prob = 0
    for i in range(sent.shape[0]-1):
        print('i:', i)
        batch_size = i+1
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        else:
            src_mask_ = src_mask[:,:]
        output_softmax = model(sent[:i+1,:], src_mask_)
        output_softmax_permuted = lnsoftmax(output_softmax.permute(1, 0, 2))
        
        print(output_softmax_permuted,output_softmax_permuted[0,i,sent[i+1,0]],output_softmax_permuted.max())
        #Index for maximum probability word
        indices = torch.argmax(output_softmax_permuted, dim=2)
        
        #Max probability word
        print('next word: ', [vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        
        prob+= output_softmax_permuted[0,i,sent[i+1,0]]
    return prob
    
    
    

In [68]:
probability(best_model,st_i)

i: 0
tensor([[[ -5.0913,  -6.2857,  -6.0924,  ..., -18.0961, -16.7388, -16.7699]]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>) tensor(-8.7135, device='cuda:0', grad_fn=<SelectBackward0>) tensor(-3.5160, device='cuda:0', grad_fn=<MaxBackward1>)
next word:  नै
i: 1
tensor([[[ -5.0913,  -6.2857,  -6.0924,  ..., -18.0961, -16.7388, -16.7699],
         [ -6.0678,  -5.9991,  -4.4271,  ..., -20.0123, -16.5251, -18.4687]]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>) tensor(-5.6694, device='cuda:0', grad_fn=<SelectBackward0>) tensor(-2.3711, device='cuda:0', grad_fn=<MaxBackward1>)
next word:  जान्छु
i: 2
tensor([[[ -5.0913,  -6.2857,  -6.0924,  ..., -18.0961, -16.7388, -16.7699],
         [ -6.0678,  -5.9991,  -4.4271,  ..., -20.0123, -16.5251, -18.4687],
         [ -6.4965,  -6.5571,  -4.5201,  ..., -19.2898, -16.4366, -16.8484]]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>) tensor(-1.0052, device='cuda:0', grad_fn=<SelectBackward0>) tensor(-1.0052, device='

tensor(-15.3881, device='cuda:0', grad_fn=<AddBackward0>)

In [27]:
# import time
# def nonnaive_generator(model: nn.Module, gen_data: Tensor, no_words = 10):
#     model.eval()
#     temp_text = text
#     src_mask = generate_square_subsequent_mask(bptt).to(device)
#     pred_text = []
#     for i in range(no_words):
#         print('i:', i)
#         batch_size = gen_data.size(0)
#         if batch_size != bptt:
#             src_mask_ = src_mask[:batch_size, :batch_size]
#         output_softmax = model(gen_data, src_mask_)
#         output_softmax_permuted = output_softmax.permute(1, 0, 2)
#         #print(softmax(output_softmax_permuted))
#         indices = torch.topk(output_softmax_permuted,10 ,dim=2,sorted=True).indices.squeeze(0)
#         values = torch.topk(softmax(output_softmax_permuted),10 ,dim=2,sorted = True).values.squeeze(0)
#         values = values/torch.sum(values,dim = 1,keepdims = True)
#         values = torch.flip(values,dims = (1,))
#         #print(output_softmax_permuted[indices])
#         print(indices,values)
#         ind_sampled = torch.distributions.Categorical(values).sample()
# #         index = indices.squeeze(0)[ind_sampled.unsqueeze(0)]
#         print('is',ind_sampled)
#         next_index = indices[-1][ind_sampled[-1]]
#         print(indices[-1][ind_sampled[-1]])
        
# #     return indices
        
#         #print(indices[0],indices[1])
#         #for j in range(batch_size):
        
#         print('next word: ', [vocab.lookup_token(next_index)],'values: ',values.squeeze(0)[-1])
                                  
# #         print(i,"Gen_data: ",gen_data,"Pred_data: ",indices)


#         pred_text.append([vocab.lookup_token((next_index))][0])
#         if(batch_size <= 10):
#             gen_data = torch.cat((gen_data[:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
#             batch_size= gen_data.size(0)
#         else:
#             gen_data = torch.cat((gen_data[1:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
#             batch_size= gen_data.size(0)
            
#     return pred_text



def nonnaive_generator(model: nn.Module, gen_data: Tensor, no_words = 5,k=50):
    model.eval()
#     temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        print('i:', i)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.topk(output_softmax_permuted,k ,dim=2).indices.squeeze(0)
        
        values = torch.topk(softmax(output_softmax_permuted),k ,dim=2).values
        values = values/torch.sum(values,dim = 2,keepdims = True)
#         values = softmax(values)
        
#         values = torch.flip(values,dims = (2,))

        
        ind_sampled = torch.distributions.Categorical(values.squeeze(0)).sample()
        next_index = indices[-1][ind_sampled[-1]]
        
        print('next token: ', tokenizer.decode([next_index]))

        print(i,"Values: ",values.squeeze(0)[-1],"Gen_data: ",gen_data,"possible tokens: ",indices[-1],"Pred_data: ",next_index)
        pred_text.append(next_index.item())
        if(batch_size < 128):
            gen_data = torch.cat((gen_data[:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
            
    return pred_text

In [39]:
#Load saved model

model.load_state_dict(torch.load('checkpoint2.pt'))
model.to(device)
best_model = model

In [66]:

print(sample_data[:,-1].unsqueeze(1))
print(sample_data)
z = generator(best_model, sample_data[:,-1].unsqueeze(1),no_words = 50)

NameError: name 'sample_data' is not defined

In [187]:
z = [t.item() for t in z]
m = splits_to_token(split_list(z))
m

['',
 'अमेरिकाको',
 'नेपाल',
 'भ्रमण',
 'गर्ने',
 'निर्णय',
 'गरेको',
 'छ',
 '।',
 'नेपाल',
 'भ्रमण']

In [188]:
'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त ' +' '.join(m)

'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त  अमेरिकाको नेपाल भ्रमण गर्ने निर्णय गरेको छ । नेपाल भ्रमण'

In [156]:
evaluate(best_model, sample_data)

1.5743927955627441

In [157]:
'''
आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , 
संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त

विकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय
'''

'\nआधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , \nसंयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त\n\nविकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय\n'

In [202]:
st = ['आधिकारिक निर्णयको कारणले']
st_i = data_process(st)

In [203]:
st_i = st_i.unsqueeze(1).to(device)

In [117]:
s = get_batch(test_data,4)


In [118]:
s

(tensor([   0,  268,  260, 1613,  266,  263,  261,  274,    0,  280,  333,  316,
          265,  263,  261,  409,  261,    0,  279,  272,  767,  261,  281,  260,
          849,  266,  268,  333,  262,  273,  269,  260,    0, 9942,  267,    0,
          262,  265,  264,  261,  274,  260,    0,    0, 1522,  261,  294,  265,
          270,    0,  268,  260,  269,  260,    0,  279,  272,  767,  261,  281,
          260,  274,    0,   31,    0, 2340,    0,  321,  260,  280,  261,  262,
            0,  258,   98,    0, 3324,  261, 1174,  272,  327,  267,    0,  279,
          272,  767,  261,  281,  260,  849,  266,  268,  333,  262,  273,  269,
          260,    0,  394,  265,  322,  266,  262,    0,  268,  260, 1613,  266,
          263,  261,  274,    0,  299,  265,  263, 2668,    0, 9942,  267,    0,
          262,  265,  264,  261,  274,  260,    0,  852]),
 tensor([ 268,  260, 1613,  266,  263,  261,  274,    0,  280,  333,  316,  265,
          263,  261,  409,  261,    0,  279,  272,

In [204]:
z_ = generator(best_model, s[0].unsqueeze(1).to(device),no_words = 50)

i: 0
torch.Size([1, 128])
next word:  à¥į
0 Gen_data:  tensor([[   0],
        [ 268],
        [ 260],
        [1613],
        [ 266],
        [ 263],
        [ 261],
        [ 274],
        [   0],
        [ 280],
        [ 333],
        [ 316],
        [ 265],
        [ 263],
        [ 261],
        [ 409],
        [ 261],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [9942],
        [ 267],
        [   0],
        [ 262],
        [ 265],
        [ 264],
        [ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
 

3 Gen_data:  tensor([[1613],
        [ 266],
        [ 263],
        [ 261],
        [ 274],
        [   0],
        [ 280],
        [ 333],
        [ 316],
        [ 265],
        [ 263],
        [ 261],
        [ 409],
        [ 261],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [9942],
        [ 267],
        [   0],
        [ 262],
        [ 265],
        [ 264],
        [ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [  

next word:  à¥ĩ
6 Gen_data:  tensor([[ 261],
        [ 274],
        [   0],
        [ 280],
        [ 333],
        [ 316],
        [ 265],
        [ 263],
        [ 261],
        [ 409],
        [ 261],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [9942],
        [ 267],
        [   0],
        [ 262],
        [ 265],
        [ 264],
        [ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [  

next word:  à¤µ
9 Gen_data:  tensor([[ 280],
        [ 333],
        [ 316],
        [ 265],
        [ 263],
        [ 261],
        [ 409],
        [ 261],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [9942],
        [ 267],
        [   0],
        [ 262],
        [ 265],
        [ 264],
        [ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 2

next word:  Ġà¤Ĺà¤°
12 Gen_data:  tensor([[ 265],
        [ 263],
        [ 261],
        [ 409],
        [ 261],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [9942],
        [ 267],
        [   0],
        [ 262],
        [ 265],
        [ 264],
        [ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 280],
        [ 261],
        [ 262],
      

...

next word:  à¤¾
36 Gen_data:  tensor([[ 262],
        [ 265],
        [ 264],
        [ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 280],
        [ 261],
        [ 262],
        [   0],
        [ 258],
        [  98],
        [   0],
        [3324],
        [ 261],
        [1174],
        [ 272],
        [ 327],
        [ 267],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 

next word:  Ġà¤¬
39 Gen_data:  tensor([[ 261],
        [ 274],
        [ 260],
        [   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 280],
        [ 261],
        [ 262],
        [   0],
        [ 258],
        [  98],
        [   0],
        [3324],
        [ 261],
        [1174],
        [ 272],
        [ 327],
        [ 267],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [ 394],
        [

next word:  à¤¾
42 Gen_data:  tensor([[   0],
        [   0],
        [1522],
        [ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 280],
        [ 261],
        [ 262],
        [   0],
        [ 258],
        [  98],
        [   0],
        [3324],
        [ 261],
        [1174],
        [ 272],
        [ 327],
        [ 267],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [ 394],
        [ 265],
        [ 322],
        [ 266],
        [ 

next word:  à¤°à¤¤
45 Gen_data:  tensor([[ 261],
        [ 294],
        [ 265],
        [ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 280],
        [ 261],
        [ 262],
        [   0],
        [ 258],
        [  98],
        [   0],
        [3324],
        [ 261],
        [1174],
        [ 272],
        [ 327],
        [ 267],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [ 394],
        [ 265],
        [ 322],
        [ 266],
        [ 262],
        [   0],
        [ 268],
       

next word:  à¤¿
48 Gen_data:  tensor([[ 270],
        [   0],
        [ 268],
        [ 260],
        [ 269],
        [ 260],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 274],
        [   0],
        [  31],
        [   0],
        [2340],
        [   0],
        [ 321],
        [ 260],
        [ 280],
        [ 261],
        [ 262],
        [   0],
        [ 258],
        [  98],
        [   0],
        [3324],
        [ 261],
        [1174],
        [ 272],
        [ 327],
        [ 267],
        [   0],
        [ 279],
        [ 272],
        [ 767],
        [ 261],
        [ 281],
        [ 260],
        [ 849],
        [ 266],
        [ 268],
        [ 333],
        [ 262],
        [ 273],
        [ 269],
        [ 260],
        [   0],
        [ 394],
        [ 265],
        [ 322],
        [ 266],
        [ 262],
        [   0],
        [ 268],
        [ 260],
        [1613],
        [ 266],
        [ 

In [205]:
st[0]+tokenizer.decode(z_)

'आधिकारिक निर्णयको कारणले्डारीले स्वागत गरेका थिए । त्यस्तै ? प्रतिनिधिसभाको बैठकमा प्रतिनिध'

In [120]:
v = [t.item() for t in s[0]] 
splits_to_token(split_list(v))

['',
 'लाईसेन्स',
 'दौडिन्छन्',
 'पुनर्वासबेलौरीमा',
 'अटो',
 'रिक्सा',
 '',
 'रञ्जित',
 'लामा',
 'पुनर्वास',
 '?',
 '२५',
 'भाद्र',
 '।',
 'कञ्चनपुरको',
 'पुनर्वासबेलौरीमा',
 'यतिखेर',
 'लाईसेन्स',
 'बिनैै',
 'अटो',
 'रिक्सा',
 'चल']

In [121]:
v_ = [t.item() for t in z_] 
splits_to_token(split_list(v_))

['ाउने',
 'गरेको',
 'छ',
 '।',
 'पुनर्वास',
 'नगरपालिकाको',
 'कार्यालय',
 'पुनर्वास',
 'नगरपाल']

In [142]:
# st = ['लामो समयसम्म प्रयोग गर्न सकिन्छ ।']
# st = ['तपाईंलाई कस्तो पुस्तकहरू मन']
st = ['नेपालमा आधुनिक']
st = ['हरेक सेपालीले']
st_i = data_process(st)
st_i = st_i.unsqueeze(1).to(device)

In [55]:
st_i.shape

torch.Size([21, 1])

In [143]:
z_ = generator(best_model, st_i,no_words =200)

i: 0
torch.Size([1, 11])
next word:  Ġà¤¸
0 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265]], device='cuda:0') Pred_data:  tensor([[271, 263, 266, 311, 280, 260, 287, 272, 274, 259, 274]],
       device='cuda:0')
i: 1
torch.Size([1, 12])
next word:  à¤¾
1 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265],
        [274]], device='cuda:0') Pred_data:  tensor([[271, 263, 266, 311, 280, 260, 287, 272, 274, 259, 274, 259]],
       device='cuda:0')
i: 2
torch.Size([1, 13])
next word:  à¤®
2 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265],
        [274],
        [259]], device='cuda:0') Pred_data:  tensor([[271, 263, 266

        [259]], device='cuda:0') Pred_data:  tensor([[271, 263, 266, 311, 280, 260, 287, 272, 274, 259, 274, 259, 268, 259,
         293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293,
         264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268]],
       device='cuda:0')
i: 29
torch.Size([1, 40])
next word:  à¤¾
29 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268]], device='cuda:0') Pred_data:  tensor([[271, 263, 26

next word:  à¤ķ
45 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264]], device='cuda:0') Pred_data:  tensor([[271, 263, 266, 311, 280, 260, 287, 272, 274, 259, 274, 259, 268, 259,
         293, 264, 26

        [264]], device='cuda:0') Pred_data:  tensor([[271, 263, 266, 311, 280, 260, 287, 272, 274, 259, 274, 259, 268, 259,
         293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293,
         264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264,
         263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263,
         489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263]],
       device='cuda:0')
i: 59
torch.Size([1, 70])
next word:  Ġà¤¸à¤ŀ
59 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],

70 Gen_data:  tensor([[324],
        [265],
        [263],
        [274],
        [265],
        [278],
        [259],
        [267],
        [272],
        [267],
        [265],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [26

...

        [263]], device='cuda:0') Pred_data:  tensor([[259, 263, 259, 291, 259, 261, 259, 293, 264, 263, 489, 260, 293, 259,
         389, 259, 274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389,
         259, 274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259,
         274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274,
         259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259,
         268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268,
         259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259,
         293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293,
         264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264,
         263, 489]], device='cuda:0')
i: 164
torch.Size([1, 128])
next word:  à¥į
164 Gen_data:  tensor([[259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263

        [259]], device='cuda:0') Pred_data:  tensor([[259, 263, 274, 260, 293, 259, 389, 259, 285, 271, 261, 259, 293, 264,
         263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263,
         489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263, 489,
         260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263, 489, 260,
         293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263, 489, 260, 293,
         259, 389, 259, 274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259,
         389, 259, 274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389,
         259, 274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259,
         274, 259, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274,
         259, 268]], device='cuda:0')
i: 172
torch.Size([1, 128])
next word:  à¤¾
172 Gen_data:  tensor([[264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259

        [293]], device='cuda:0') Pred_data:  tensor([[261, 311, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274,
         311, 268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259,
         268, 259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268,
         259, 293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259,
         293, 264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293,
         264, 263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264,
         263, 489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263,
         489, 260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263, 489,
         260, 293, 259, 389, 259, 274, 259, 268, 259, 293, 264, 263, 489, 260,
         293, 259]], device='cuda:0')
i: 180
torch.Size([1, 128])
next word:  à¤²à¤®
180 Gen_data:  tensor([[274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [

torch.Size([1, 128])
next word:  à¤ķ
188 Gen_data:  tensor([[260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
   

next word:  à¤¾
196 Gen_data:  tensor([[259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [268],
        [259],
        [293],
        [264],
        [263],
        [489],
        [260],
        [293],
        [259],
        [389],
        [259],
        [274],
        [259],
        [

In [144]:
z_ = [t.item() for t in z_]
m = splits_to_token(split_list(z_))
m

[' सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाज']

In [145]:
' '.join(st) + ' '.join(m)

'हरेक सेपालीले सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाजिक सञ्जालमा सामाज'

In [154]:

st_i

tensor([[268],
        [292],
        [259],
        [346],
        [292],
        [260],
        [683],
        [296],
        [260],
        [262]], device='cuda:0')

In [155]:
z_ = generator(best_model, st_i,no_words = 50)

i: 0
next word:  े
0 Gen_data:  tensor([[268],
        [292],
        [259],
        [346],
        [292],
        [260],
        [683],
        [296],
        [260],
        [262]], device='cuda:0') Pred_data:  tensor([[259, 259, 346, 272, 260, 683, 296, 260, 262, 265]], device='cuda:0')
i: 1
next word:  र
1 Gen_data:  tensor([[268],
        [292],
        [259],
        [346],
        [292],
        [260],
        [683],
        [296],
        [260],
        [262],
        [265]], device='cuda:0') Pred_data:  tensor([[259, 259, 346, 272, 260, 683, 296, 260, 262, 265, 296]],
       device='cuda:0')
i: 2
next word:  े
2 Gen_data:  tensor([[268],
        [292],
        [259],
        [346],
        [292],
        [260],
        [683],
        [296],
        [260],
        [262],
        [265],
        [296]], device='cuda:0') Pred_data:  tensor([[259, 259, 346, 272, 260, 683, 296, 260, 262, 265, 296, 265]],
       device='cuda:0')
i: 3
next word:  क
3 Gen_data:  tensor([[268],
        [

next word:  ि
38 Gen_data:  tensor([[263],
        [311],
        [263],
        [269],
        [265],
        [264],
        [266],
        [300],
        [281],
        [276],
        [260],
        [346],
        [263],
        [270],
        [266],
        [284]], device='cuda:0') Pred_data:  tensor([[262, 263, 267, 265, 264, 266, 300, 281, 276, 260, 346, 263, 270, 266,
         284, 263]], device='cuda:0')
i: 39
next word:  त
39 Gen_data:  tensor([[311],
        [263],
        [269],
        [265],
        [264],
        [266],
        [300],
        [281],
        [276],
        [260],
        [346],
        [263],
        [270],
        [266],
        [284],
        [263]], device='cuda:0') Pred_data:  tensor([[263, 267, 265, 264, 266, 300, 281, 276, 260, 346, 263, 270, 266, 284,
         263, 269]], device='cuda:0')
i: 40
next word:  ा
40 Gen_data:  tensor([[263],
        [269],
        [265],
        [264],
        [266],
        [300],
        [281],
        [276],
        [2

In [157]:
' '.join(st)+' ' +''.join(z_)

'म भारत भ्रमण गर्न ेरेकोछ।प्तियोगिताकोपाधिजितेकोछ।प्तियोगिताकोपाधिजित'

In [78]:
 x = torch.arange(1., 6.)
j = torch.topk(x, 3)

In [79]:
j.values

tensor([5., 4., 3.])

In [80]:
vocab['?']

2

In [141]:
bptt = 32

In [155]:
z__ = nonnaive_generator(best_model, st_i,no_words = 100,k=200)

i: 0
next token:   समस
0 Values:  tensor([0.0849, 0.0637, 0.0512, 0.0475, 0.0442, 0.0333, 0.0325, 0.0323, 0.0314,
        0.0234, 0.0226, 0.0222, 0.0205, 0.0196, 0.0196, 0.0186, 0.0176, 0.0164,
        0.0158, 0.0149, 0.0146, 0.0134, 0.0131, 0.0130, 0.0100, 0.0099, 0.0099,
        0.0094, 0.0084, 0.0072, 0.0071, 0.0071, 0.0056, 0.0053, 0.0053, 0.0050,
        0.0043, 0.0040, 0.0039, 0.0038, 0.0038, 0.0038, 0.0037, 0.0037, 0.0035,
        0.0035, 0.0034, 0.0031, 0.0031, 0.0028, 0.0027, 0.0027, 0.0027, 0.0026,
        0.0026, 0.0026, 0.0026, 0.0025, 0.0024, 0.0023, 0.0023, 0.0023, 0.0022,
        0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0021, 0.0021, 0.0021, 0.0020,
        0.0020, 0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018,
        0.0017, 0.0017, 0.0016, 0.0016, 0.0016, 0.0015, 0.0015, 0.0015, 0.0015,
        0.0015, 0.0014, 0.0014, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013,
        0.0013, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012,
      

next token:  ा
5 Values:  tensor([9.9787e-01, 1.2271e-03, 3.2225e-04, 1.9628e-04, 1.1616e-04, 9.5273e-05,
        4.9151e-05, 3.0784e-05, 1.4832e-05, 1.4134e-05, 1.2723e-05, 3.1031e-06,
        2.7852e-06, 2.6645e-06, 2.6323e-06, 2.5878e-06, 2.1338e-06, 2.1233e-06,
        1.7228e-06, 1.6472e-06, 1.5433e-06, 1.5234e-06, 1.3820e-06, 1.3551e-06,
        1.2500e-06, 9.0003e-07, 8.5903e-07, 7.8956e-07, 6.8264e-07, 6.5768e-07,
        6.1235e-07, 5.9307e-07, 5.7827e-07, 5.7156e-07, 5.6254e-07, 5.4540e-07,
        5.4357e-07, 5.2389e-07, 4.8620e-07, 4.6556e-07, 4.5048e-07, 4.4445e-07,
        4.4057e-07, 4.3434e-07, 3.6648e-07, 3.5053e-07, 3.0926e-07, 3.0575e-07,
        2.9911e-07, 2.8918e-07, 2.8565e-07, 2.7954e-07, 2.7627e-07, 2.7515e-07,
        2.7055e-07, 2.6943e-07, 2.6549e-07, 2.6326e-07, 2.5240e-07, 2.4170e-07,
        2.3897e-07, 2.2720e-07, 2.1876e-07, 2.1622e-07, 2.0390e-07, 1.9610e-07,
        1.8839e-07, 1.7740e-07, 1.6489e-07, 1.5758e-07, 1.5532e-07, 1.5237e-07,
        1.5219

        [296]], device='cuda:0') possible tokens:  tensor([ 260,  265,  264,  259,  272,  399,  546,  271,  266,  493,  332,  936,
         286,  945,  318,  302,  296,  522,  281,  311,  347,  330,  340,  285,
        1235,  420,  274,  282,  288,  307,  276,  304,  295, 2209,  297, 2548,
         289,  291,  220,  444, 1813,  263,  312, 1736,  310,  303,  335, 2196,
         474,  325,  292,  410,  283,  343,  345,  323,  381,  300,  508,  262,
        3432,  268,  339,  597,  261,  349, 1714,  333,  334,  329,  757, 1581,
         368, 2236, 2615,  591,  542,  380,  197,  273,  267, 1321,  390,  460,
         374,  543,  401,  413,  278,  375,  371,  402,  636,  800, 1782,  403,
        2438,  398,  423, 1096,  670,  364,  344,  385,  409,  651, 1746,  419,
        1241,  466,  387, 2918,  369, 1601, 1927,  438,  365,  384, 1141, 3957,
         320,  270, 1587,  404, 1120,  464,  515,  517, 1025, 3035,  317,  293,
        4550,  367,  397,  579,  752,  361,  353,  405,  745,  695,  

next token:  छ
17 Values:  tensor([0.1084, 0.0783, 0.0742, 0.0669, 0.0562, 0.0521, 0.0463, 0.0421, 0.0333,
        0.0247, 0.0237, 0.0205, 0.0204, 0.0191, 0.0184, 0.0141, 0.0139, 0.0139,
        0.0135, 0.0119, 0.0108, 0.0099, 0.0096, 0.0080, 0.0077, 0.0075, 0.0068,
        0.0068, 0.0066, 0.0066, 0.0065, 0.0064, 0.0054, 0.0052, 0.0052, 0.0051,
        0.0049, 0.0048, 0.0048, 0.0047, 0.0035, 0.0033, 0.0032, 0.0031, 0.0031,
        0.0030, 0.0028, 0.0024, 0.0023, 0.0020, 0.0019, 0.0018, 0.0018, 0.0018,
        0.0018, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0015, 0.0014, 0.0014,
        0.0014, 0.0013, 0.0013, 0.0013, 0.0013, 0.0012, 0.0012, 0.0012, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0010, 0.0010, 0.0009, 0.0009, 0.0009,
        0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008, 0.0007, 0.0007,
        0.0007, 0.0007, 0.0007, 0.0007, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006,
        0.0006, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.000

next token:  क
23 Values:  tensor([1.5007e-01, 7.5154e-02, 7.3775e-02, 6.9427e-02, 6.1413e-02, 5.4064e-02,
        3.5775e-02, 3.0720e-02, 2.9861e-02, 2.6355e-02, 2.6219e-02, 2.5545e-02,
        2.5045e-02, 2.4566e-02, 2.4410e-02, 2.2925e-02, 2.2599e-02, 1.7102e-02,
        1.2422e-02, 1.1478e-02, 1.1321e-02, 1.0068e-02, 9.5948e-03, 9.2758e-03,
        9.1303e-03, 7.5811e-03, 5.2729e-03, 4.4213e-03, 3.9065e-03, 3.8681e-03,
        3.6545e-03, 3.6516e-03, 3.5674e-03, 3.2509e-03, 3.2477e-03, 3.0910e-03,
        2.8163e-03, 2.4020e-03, 2.3315e-03, 2.2839e-03, 2.1657e-03, 2.1644e-03,
        2.1228e-03, 2.0114e-03, 1.7949e-03, 1.7256e-03, 1.6285e-03, 1.5352e-03,
        1.2813e-03, 1.2391e-03, 1.2300e-03, 1.1857e-03, 1.1656e-03, 1.1448e-03,
        1.1359e-03, 1.0483e-03, 1.0470e-03, 1.0191e-03, 1.0049e-03, 9.6293e-04,
        9.3251e-04, 8.9243e-04, 8.8563e-04, 8.7341e-04, 8.3280e-04, 8.2555e-04,
        8.1896e-04, 8.1658e-04, 7.9902e-04, 7.1351e-04, 7.0048e-04, 6.9738e-04,
        6.743

...

        0.0001, 0.0001], device='cuda:0', grad_fn=<SelectBackward0>) Gen_data:  tensor([[ 324],
        [ 265],
        [ 263],
        [ 274],
        [ 265],
        [ 278],
        [ 259],
        [ 267],
        [ 272],
        [ 267],
        [ 265],
        [ 558],
        [ 260],
        [ 270],
        [ 259],
        [ 325],
        [ 259],
        [ 301],
        [ 259],
        [ 262],
        [ 296],
        [ 260],
        [ 262],
        [ 271],
        [ 359],
        [ 260],
        [ 262],
        [ 265],
        [ 317],
        [ 281],
        [ 274],
        [ 265],
        [ 280],
        [ 259],
        [ 263],
        [ 259],
        [ 280],
        [ 260],
        [ 270],
        [ 259],
        [ 278],
        [ 259],
        [ 261],
        [ 272],
        [ 324],
        [ 271],
        [ 267],
        [ 259],
        [ 327],
        [ 288],
        [ 265],
        [ 278],
        [ 259],
        [ 372],
        [ 259],
        [ 289],
        [ 264],
        

next token:  ्
82 Values:  tensor([7.6589e-01, 1.4915e-01, 3.7691e-02, 2.6128e-02, 1.4518e-02, 1.7544e-03,
        1.5055e-03, 8.0337e-04, 5.7660e-04, 4.8143e-04, 4.0352e-04, 1.8033e-04,
        1.7665e-04, 1.3106e-04, 4.3043e-05, 4.1745e-05, 4.0315e-05, 3.5001e-05,
        3.4492e-05, 2.9288e-05, 2.8605e-05, 2.6113e-05, 2.3192e-05, 2.1843e-05,
        1.7126e-05, 1.5011e-05, 1.0309e-05, 9.9662e-06, 9.5874e-06, 9.4889e-06,
        8.9850e-06, 8.6783e-06, 8.4464e-06, 7.3645e-06, 7.1415e-06, 6.9890e-06,
        6.8004e-06, 6.2719e-06, 5.6783e-06, 5.5999e-06, 5.1668e-06, 5.0470e-06,
        4.6498e-06, 4.5156e-06, 3.1842e-06, 3.1676e-06, 3.1250e-06, 2.9905e-06,
        2.6021e-06, 2.5644e-06, 2.4498e-06, 2.4350e-06, 2.4144e-06, 2.2606e-06,
        2.1934e-06, 2.1793e-06, 2.1381e-06, 2.0224e-06, 1.9866e-06, 1.9545e-06,
        1.8770e-06, 1.8644e-06, 1.7398e-06, 1.6929e-06, 1.6877e-06, 1.5986e-06,
        1.5249e-06, 1.4442e-06, 1.4438e-06, 1.3861e-06, 1.2834e-06, 1.2440e-06,
        1.237

87 Values:  tensor([9.9010e-01, 5.9005e-03, 2.6083e-03, 2.5238e-04, 1.3220e-04, 8.2307e-05,
        6.8259e-05, 5.8547e-05, 5.0216e-05, 4.3862e-05, 4.1971e-05, 3.5992e-05,
        2.4861e-05, 2.4274e-05, 2.3936e-05, 2.0933e-05, 1.9123e-05, 1.9017e-05,
        1.8656e-05, 1.7597e-05, 1.6847e-05, 1.4123e-05, 1.4018e-05, 1.1958e-05,
        1.1877e-05, 1.1334e-05, 1.1278e-05, 1.1267e-05, 9.9323e-06, 9.7087e-06,
        9.5757e-06, 9.0630e-06, 8.2486e-06, 8.1662e-06, 8.1090e-06, 7.6142e-06,
        7.2709e-06, 6.8997e-06, 6.5515e-06, 6.3867e-06, 6.0969e-06, 5.9665e-06,
        4.8837e-06, 4.7062e-06, 4.6087e-06, 4.3654e-06, 4.3420e-06, 4.1473e-06,
        3.9765e-06, 3.9568e-06, 3.9521e-06, 3.9502e-06, 3.8914e-06, 3.8746e-06,
        3.7278e-06, 3.7183e-06, 3.6830e-06, 3.6685e-06, 3.6486e-06, 3.4727e-06,
        3.2717e-06, 3.2165e-06, 3.1552e-06, 3.1261e-06, 3.0814e-06, 3.0743e-06,
        3.0537e-06, 2.9946e-06, 2.9086e-06, 2.8864e-06, 2.7165e-06, 2.6852e-06,
        2.6359e-06, 2.6156e-

92 Values:  tensor([2.2759e-01, 9.7857e-02, 6.4821e-02, 4.2415e-02, 3.9846e-02, 3.0705e-02,
        2.8295e-02, 2.7277e-02, 2.6554e-02, 2.6500e-02, 2.5781e-02, 2.3924e-02,
        2.0607e-02, 1.9954e-02, 1.8584e-02, 1.8536e-02, 1.8281e-02, 1.7304e-02,
        1.3529e-02, 1.2794e-02, 1.1678e-02, 1.1423e-02, 1.1077e-02, 1.0093e-02,
        9.3020e-03, 9.0894e-03, 8.6232e-03, 8.2234e-03, 7.8163e-03, 7.5351e-03,
        5.2395e-03, 5.2150e-03, 5.1971e-03, 3.4710e-03, 3.3081e-03, 3.1394e-03,
        2.8379e-03, 2.7440e-03, 2.0990e-03, 2.0527e-03, 2.0351e-03, 1.9056e-03,
        1.8919e-03, 1.8796e-03, 1.8766e-03, 1.7376e-03, 1.7337e-03, 1.5846e-03,
        1.4378e-03, 1.4153e-03, 1.4053e-03, 1.3209e-03, 1.2813e-03, 1.2287e-03,
        1.0484e-03, 1.0001e-03, 9.6971e-04, 9.6302e-04, 9.5550e-04, 9.5334e-04,
        9.3110e-04, 9.1484e-04, 8.8685e-04, 8.7840e-04, 8.6066e-04, 8.2514e-04,
        8.2304e-04, 7.8493e-04, 7.7468e-04, 7.0544e-04, 6.5487e-04, 6.4584e-04,
        6.2681e-04, 5.7406e-

next token:  ्
98 Values:  tensor([9.8097e-01, 9.9669e-03, 4.4853e-03, 1.3717e-03, 5.7940e-04, 3.8140e-04,
        2.9014e-04, 2.8527e-04, 2.8239e-04, 2.4796e-04, 1.1973e-04, 1.0688e-04,
        5.5649e-05, 4.6397e-05, 4.2158e-05, 4.1922e-05, 3.8497e-05, 3.2158e-05,
        2.8837e-05, 2.8779e-05, 2.7419e-05, 2.5064e-05, 2.3009e-05, 2.2449e-05,
        1.9311e-05, 1.8355e-05, 1.7197e-05, 1.7142e-05, 1.6596e-05, 1.6445e-05,
        1.2900e-05, 1.1843e-05, 1.1308e-05, 1.1188e-05, 1.0680e-05, 1.0543e-05,
        9.2143e-06, 9.0522e-06, 9.0263e-06, 8.4413e-06, 8.3783e-06, 8.3576e-06,
        7.8785e-06, 7.8710e-06, 7.8533e-06, 7.8412e-06, 7.2451e-06, 6.8588e-06,
        6.7725e-06, 6.3965e-06, 6.1989e-06, 6.1835e-06, 6.1057e-06, 5.3300e-06,
        5.2734e-06, 5.2314e-06, 5.1738e-06, 5.1275e-06, 5.0424e-06, 3.9732e-06,
        3.9267e-06, 3.7103e-06, 3.6639e-06, 3.6425e-06, 3.5706e-06, 3.5100e-06,
        3.4687e-06, 3.4073e-06, 3.0996e-06, 2.9229e-06, 2.8183e-06, 2.6179e-06,
        2.577

In [147]:
# m = splits_to_token(split_list(z__))

In [156]:
m = splits_to_token(split_list(z__))

In [157]:
' '.join(st)+ ' '.join(m)

'हरेक सेपालीले समस्या समाधान गर्नुपर्नेछ । सेवाकाव्यापारीहरुलाइ नेपालका बिक्रेता खाएर केहि हदसम्म सन्दर्भमा जाने समस्या समाधान गर्नेलाई योजना ? वेवास्त'

In [149]:
# z__ = [t.item() for t in z__]
# m = splits_to_token(split_list(z__))
# m

In [140]:
m

[' निर्माणका लागि कार्यालयमा सरकारले दुई हजार रुपैयाँ पुग्ने गरेको छ । त्यसमाथिको राजनीतिप्रति मैले सर्बसाधारणलाई प्रत्येक अस्पतालको लापरवाहीका']

In [83]:
vocab['।']

1

In [74]:
z__

[636,
 264,
 281,
 323,
 266,
 276,
 260,
 326,
 259,
 326,
 266,
 304,
 265,
 309,
 295,
 265,
 261,
 260,
 262,
 271,
 344,
 260,
 279,
 286,
 262,
 281,
 295,
 259,
 268,
 260,
 261,
 259,
 297,
 264,
 301,
 264,
 263,
 266,
 276,
 260,
 346,
 264,
 280,
 259,
 279,
 285,
 556,
 266,
 284,
 310,
 271,
 287,
 259,
 338,
 271,
 323,
 266,
 297,
 260,
 336,
 260,
 269,
 264,
 269,
 1859,
 286,
 288,
 264,
 313,
 600,
 259,
 351,
 282,
 259,
 268,
 296,
 260,
 317,
 399,
 281,
 323,
 266,
 274,
 311,
 270,
 266,
 505,
 259,
 313,
 385,
 259,
 270,
 399,
 285,
 538,
 340,
 274,
 271,
 273,
 260]